## Cats vs Dogs
For this workshop you will be building a Convolutional neural network to classify cats vs dogs. You will need to be familiar with the theory of CNNs. Visit our lesson [here](http://caisplusplus.usc.edu/blog/curriculum/lesson7) for more info.

In [11]:
# Imports, make sure you have cv2 installed!
import os
import numpy as np
from scipy.ndimage import imread
import cv2
import sklearn.utils

# You are not allowed to change any of these constants.

INPUT_STRING = 'burrito'
DATA_PATH_1 = 'data_generation/training/positive_samples/'
DATA_PATH_2 = 'data_generation/training/false_samples/'
TEST_PERCENT = 0.1
SELECT_SUBSET_PERCENT = 0.15

# The cat and dog images are of variable size.
RESIZE_WIDTH=32
RESIZE_HEIGHT=32
EPOCHS = 10

In [12]:
# Lets get started by loading the data.
# Make sure you have the data downloaded to ./data
# To download the data go to https://www.kaggle.com/c/dogs-vs-cats/data and download train.zip

X = []
Y = []

files_1 = os.listdir(DATA_PATH_1)
# Shuffle so we are selecting about an equal number of dog and cat images.
shuffled_files_1 = sklearn.utils.shuffle(files_1)
select_count_1 = int(len(shuffled_files_1) * SELECT_SUBSET_PERCENT)

print('Going to load %i files' % select_count_1)

subset_files_select_1 = shuffled_files_1[:select_count_1]

DISPLAY_COUNT = 1000

for i, input_file in enumerate(subset_files_select_1):
    if i % DISPLAY_COUNT == 0 and i != 0:
        print('Have loaded %i samples' % i)
        
    img = imread(DATA_PATH_1 + input_file)
    img = cv2.resize(img, (RESIZE_WIDTH, RESIZE_HEIGHT), interpolation=cv2.INTER_CUBIC)
    X.append(img)
    Y.append(0.0)
    

files_2 = os.listdir(DATA_PATH_2)
# Shuffle so we are selecting about an equal number of dog and cat images.
shuffled_files_2 = sklearn.utils.shuffle(files_2)
select_count_2 = int(len(shuffled_files_2) * SELECT_SUBSET_PERCENT)

print('Going to load %i files' % select_count_2)

subset_files_select_2 = shuffled_files_2[:select_count_2]
    
for i, input_file in enumerate(subset_files_select_2):
    if i % DISPLAY_COUNT == 0 and i != 0:
        print('Have loaded %i samples' % i)
        
    img = imread(DATA_PATH_2 + input_file)
    img = cv2.resize(img, (RESIZE_WIDTH, RESIZE_HEIGHT), interpolation=cv2.INTER_CUBIC)
    X.append(img)
    Y.append(1.0)
        
X = np.array(X)
Y = np.array(Y)

test_size = int(len(X) * TEST_PERCENT)

test_X = X[:test_size]
test_Y = Y[:test_size]
train_X = X[test_size:]
train_Y = Y[test_size:]

print('Train set has dimensionality %s' % str(train_X.shape))
print('Test set has dimensionality %s' % str(test_X.shape))

# Apply some normalization here.
train_X = train_X.astype('float32')
test_X = test_X.astype('float32')
train_X /= 255
test_X /= 255

Going to load 1200 files
Have loaded 1000 samples
Going to load 1200 files
Have loaded 1000 samples
Train set has dimensionality (2160, 32, 32, 3)
Test set has dimensionality (240, 32, 32, 3)


In [13]:
######################################
#TODO: (Optional)
# Perform any data preprocessing steps



######################################

### Defining the network
Here are some useful resources to help with defining a powerful network.
- Convolution layers (use the 2D convolution) https://keras.io/layers/convolutional/
- Batch norm layer https://keras.io/layers/normalization/
- Layer initializers https://keras.io/initializers/
- Dense layer https://keras.io/layers/core/#dense
- Activation functions https://keras.io/layers/core/#activation
- Regulizers: 
    - https://keras.io/layers/core/#dropout
    - https://keras.io/regularizers/
    - https://keras.io/callbacks/#earlystopping
    - https://keras.io/constraints/

In [14]:
######################################
#TODO:
# Import necessary layers.
from keras.models import Sequential
from keras.layers import Input, Dropout, Flatten, Conv2D, MaxPooling2D, Dense, Activation
from keras.optimizers import RMSprop
from keras.layers.normalization import BatchNormalization
from keras.constraints import max_norm
######################################

model = Sequential()

######################################
#TODO:
# Define the network
model.add(Conv2D(32, (3, 3), padding='same', kernel_initializer='glorot_normal', input_shape=(RESIZE_WIDTH, RESIZE_HEIGHT, 3)))
#model.add(BatchNormalization(axis=-1))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3), padding='same', kernel_initializer='glorot_normal'))
#model.add(BatchNormalization(axis=-1))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(64,(3, 3), padding='same', kernel_initializer='glorot_normal'))
#model.add(BatchNormalization(axis=-1))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3), padding='same', kernel_initializer='glorot_normal'))
#model.add(BatchNormalization(axis=-1))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())

# Fully connected layer
model.add(Dense(512, kernel_initializer='glorot_normal'))
#model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1, kernel_initializer='glorot_normal'))
model.add(Activation('sigmoid'))
######################################


######################################
#TODO:
# Define your loss and your objective
optimizer = RMSprop(lr=1e-4)
loss = 'binary_crossentropy'
model.compile(loss=loss, optimizer=optimizer, metrics=['accuracy'])
######################################

## Test Time
Now it's time to actually test the network. Don't change any of the parameters here except for the batch size.

Get above **65%**!

In [ ]:
######################################
#TODO:
# Define the batch size
batch_size = 64
######################################
model.fit(train_X, train_Y, batch_size=batch_size, epochs=EPOCHS, validation_split=0.2, verbose=1, shuffle=True)

Train on 1728 samples, validate on 432 samples
Epoch 1/10
1728/1728 [==============================] - 7s 4ms/step - loss: 0.6635 - acc: 0.5862 - val_loss: 0.6647 - val_acc: 0.8171
Epoch 2/10
1728/1728 [==============================] - 6s 3ms/step - loss: 0.6069 - acc: 0.6580 - val_loss: 0.6087 - val_acc: 0.8843
Epoch 3/10
1728/1728 [==============================] - 6s 4ms/step - loss: 0.5179 - acc: 0.7581 - val_loss: 0.4251 - val_acc: 0.9769
Epoch 4/10
1728/1728 [==============================] - 6s 4ms/step - loss: 0.4259 - acc: 0.8171 - val_loss: 0.3757 - val_acc: 0.9352
Epoch 5/10
1728/1728 [==============================] - 6s 4ms/step - loss: 0.3626 - acc: 0.8385 - val_loss: 0.1328 - val_acc: 0.9977
Epoch 6/10
1728/1728 [==============================] - 6s 3ms/step - loss: 0.3254 - acc: 0.8553 - val_loss: 0.3721 - val_acc: 0.8403
Epoch 7/10
1728/1728 [==============================] - 7s 4ms/step - loss: 0.2874 - acc: 0.8941 - val_loss: 0.3314 - val_acc: 0.8727
Epoch 8/10
 128

In [ ]:
loss, acc = model.evaluate(test_X, test_Y, batch_size=batch_size, verbose=1)

print('')
print('Got %.2f%% accuracy' % (acc * 100.))